# Python Notebook for remote sensing work
Repository made for Artificial Intelligence & Neural Network course with Prof. Ciarfuglia @ Sapienza 

Import necessary items with this:

In [5]:
# Let's import our own modules
import sys
sys.path.insert(0, './src')
import train
import utils
import dataset
import model

import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms

We should test that the CUDA platform has been successfully recognized and it's being in use with pyTorch

In [6]:
device = utils.set_and_test_cuda()

Using PyTorch version: 2.3.0+cu121  Device: cpu


SystemExit: CUDA not available.  Exiting.

/home/ricky/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Torch has a very useful way of cleaning the dataset, normalizing it and reshaping as needed, using the function classes available in torchvision.transforms()
We will be using them to transform our images into Tensors too

In [ ]:
transform = transforms.transforms([
    transforms.Normalize(mean=0, std=0.1),
    transforms.ToTensor()
])


# Create a dataset class

Now we'll need to create a SN6Dataset derived from data.Dataset that will import the data and return the data and labels, and that implements __len__ and __getitem__ methods
We'll be using RasterIO

We can see the implementation of the dataset in src/dataset.py

In [4]:
train_dataset = dataset.SN6Dataset()

# DataLoader class

Splitto il dato in train, validation e test set, ma quello lo faccio io a mano


Now that we have a dataset that imports everything to a class we need to use it for our purposes

# Create a dataset class

Now we'll need to create a SN6Dataset derived from data.Dataset that will import the data and return the data and labels, and that implements __len__ and __getitem__ methods



In [ ]:

data_loader = data.DataLoader(dataset_train, batch_size=10, shuffle=True)